In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/csiro-biomasszipdataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
#import pandas as pd
#df = pd.read_csv('C:\Users\hp pc\OneDrive\Desktop\CSIRO_bisomass_agent\data\sample_submission.csv')
#df.head()


In [5]:

#df = pd.read_csv('/kaggle/input/csiro-biomasszipdataset/train.csv')
#df.head()


In [6]:
#df = pd.read_csv('/kaggle/input/csiro-biomasszipdataset/test.csv')
#df.head()

In [8]:
import os
import pandas as pd

def load_or_make_data():
    # Fixed Kaggle dataset paths
    TRAIN_PATH = "/kaggle/input/csiro-biomasszipdataset/train.csv"
    TEST_PATH  = "/kaggle/input/csiro-biomasszipdataset/test.csv"

    # Check if train.csv exists
    if not os.path.exists(TRAIN_PATH):
        raise FileNotFoundError(f"❌ train.csv NOT found at: {TRAIN_PATH}\n"
                                f"Please verify the dataset is added in Kaggle Notebook.")

    # Check if test.csv exists
    if not os.path.exists(TEST_PATH):
        raise FileNotFoundError(f"❌ test.csv NOT found at: {TEST_PATH}\n"
                                f"Please verify the dataset is added in Kaggle Notebook.")

    print(f"✅ Loading TRAIN data from: {TRAIN_PATH}")
    train_df = pd.read_csv(TRAIN_PATH)

    print(f"✅ Loading TEST data from: {TEST_PATH}")
    test_df = pd.read_csv(TEST_PATH)

    # Return ONLY real competition data
    return train_df, test_df


In [10]:
#train_df, test_df = load_or_make_data()

#train_df.head(), test_df.head()


In [7]:
# ============================================================
# 1. Imports & Config
# ============================================================
import os
import gc
import numpy as np
import pandas as pd

from tqdm import tqdm

# ML / preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from sklearn.ensemble import VotingRegressor

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# PyTorch / CNN
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as T
import torchvision.models as models

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)

print("✅ Imports done. Torch CUDA available:", torch.cuda.is_available())


✅ Imports done. Torch CUDA available: False


In [8]:
# ============================================================
# 2. Load REAL competition data (NO synthetic data)
# ============================================================
def load_data():
    TRAIN_PATH = "/kaggle/input/csiro-biomasszipdataset/train.csv"
    TEST_PATH  = "/kaggle/input/csiro-biomasszipdataset/test.csv"

    if not os.path.exists(TRAIN_PATH):
        raise FileNotFoundError(f"❌ train.csv not found at: {TRAIN_PATH}")
    if not os.path.exists(TEST_PATH):
        raise FileNotFoundError(f"❌ test.csv not found at: {TEST_PATH}")

    print(f"✅ Loading train from: {TRAIN_PATH}")
    print(f"✅ Loading test  from: {TEST_PATH}")

    train_df = pd.read_csv(TRAIN_PATH)
    test_df  = pd.read_csv(TEST_PATH)

    print("Train shape:", train_df.shape)
    print("Test  shape:", test_df.shape)
    return train_df, test_df

train_df, test_df = load_data()
display(train_df.head())
display(test_df.head())


✅ Loading train from: /kaggle/input/csiro-biomasszipdataset/train.csv
✅ Loading test  from: /kaggle/input/csiro-biomasszipdataset/test.csv
Train shape: (1785, 9)
Test  shape: (5, 3)


,sample_id,image_path,Sampling_Date,State,Species,Pre_GSHH_NDVI,Height_Ave_cm,target_name,target
0,ID1011485656__Dry_Clover_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Clover_g,0.0000
1,ID1011485656__Dry_Dead_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Dead_g,31.9984
2,ID1011485656__Dry_Green_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Green_g,16.2751
3,ID1011485656__Dry_Total_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,Dry_Total_g,48.2735
4,ID1011485656__GDM_g,train/ID1011485656.jpg,2015/9/4,Tas,Ryegrass_Clover,0.62,4.6667,GDM_g,16.2750


,sample_id,image_path,target_name
0,ID1001187975__Dry_Clover_g,test/ID1001187975.jpg,Dry_Clover_g
1,ID1001187975__Dry_Dead_g,test/ID1001187975.jpg,Dry_Dead_g
2,ID1001187975__Dry_Green_g,test/ID1001187975.jpg,Dry_Green_g
3,ID1001187975__Dry_Total_g,test/ID1001187975.jpg,Dry_Total_g
4,ID1001187975__GDM_g,test/ID1001187975.jpg,GDM_g


In [9]:
# ============================================================
# 3. Image path resolution
#    - Build absolute paths for images in Kaggle environment
# ============================================================

def build_image_paths(df, dataset_roots, image_col="image_path", new_col="image_abs_path", n_check=50):
    """
    Try multiple root folders and path patterns to find which combination
    gives the most existing image files. Then apply that logic to the whole df.
    """
    if image_col not in df.columns:
        raise ValueError(f"'{image_col}' column is missing from dataframe.")

    # We'll inspect a subset for speed
    sample_df = df[[image_col]].dropna().head(n_check).copy()
    sample_paths = sample_df[image_col].tolist()

    # Candidate (root, pattern) combos
    # pattern = lambda root, p: root + something + p
    def pat1(root, p):  # join directly
        return os.path.join(root, p)
    def pat2(root, p):  # assume 'train/' (for original competition style)
        return os.path.join(root, "train", p)
    def pat3(root, p):  # assume CSV has "train/IDxxxx.jpg" already
        return os.path.join(root, p.lstrip("/"))

    patterns = [pat1, pat2, pat3]

    best_root = None
    best_pat = None
    best_count = -1

    print("🔍 Searching for best image root+pattern combination...")
    for root in dataset_roots:
        if not os.path.exists(root):
            continue
        for fn in patterns:
            cnt = 0
            for p in sample_paths:
                candidate = fn(root, p)
                if os.path.exists(candidate):
                    cnt += 1
            print(f"  Root: {root}, pattern: {fn.__name__}, exists count: {cnt}")
            if cnt > best_count:
                best_count = cnt
                best_root = root
                best_pat = fn

    if best_root is None or best_count == 0:
        print("⚠️ Could not find any valid image path combination. CNN will be skipped.")
        df[new_col] = np.nan
        return df, None, None

    print(f"\n✅ Best combination → root: {best_root}, pattern: {best_pat.__name__} "
          f"(found {best_count}/{len(sample_paths)} in sample)")

    # Apply to full df
    abs_paths = []
    for p in df[image_col].fillna(""):
        if p == "":
            abs_paths.append("")
        else:
            abs_paths.append(best_pat(best_root, p))
    df[new_col] = abs_paths

    # Show a few examples
    print("\n🔎 Example image_abs_path values:")
    print(df[new_col].head())

    # Quick existence check
    print("\n🔎 os.path.exists check for first 5 images:")
    for p in df[new_col].head():
        print(p, "->", os.path.exists(p))

    return df, best_root, best_pat


# Candidate roots you likely have in this Kaggle environment
candidate_roots = [
    "/kaggle/input/csiro-biomasszipdataset",   # your zipfile dataset root
    "/kaggle/input/csiro-biomass",          # original competition dataset
]

train_df, best_root, best_pat = build_image_paths(train_df, candidate_roots)
test_df,  _,        _        = build_image_paths(test_df, candidate_roots)


🔍 Searching for best image root+pattern combination...
  Root: /kaggle/input/csiro-biomasszipdataset, pattern: pat1, exists count: 50
  Root: /kaggle/input/csiro-biomasszipdataset, pattern: pat2, exists count: 0
  Root: /kaggle/input/csiro-biomasszipdataset, pattern: pat3, exists count: 50

✅ Best combination → root: /kaggle/input/csiro-biomasszipdataset, pattern: pat1 (found 50/50 in sample)

🔎 Example image_abs_path values:
0    /kaggle/input/csiro-biomasszipdataset/train/ID...
1    /kaggle/input/csiro-biomasszipdataset/train/ID...
2    /kaggle/input/csiro-biomasszipdataset/train/ID...
3    /kaggle/input/csiro-biomasszipdataset/train/ID...
4    /kaggle/input/csiro-biomasszipdataset/train/ID...
Name: image_abs_path, dtype: object

🔎 os.path.exists check for first 5 images:
/kaggle/input/csiro-biomasszipdataset/train/ID1011485656.jpg -> True
/kaggle/input/csiro-biomasszipdataset/train/ID1011485656.jpg -> True
/kaggle/input/csiro-biomasszipdataset/train/ID1011485656.jpg -> True
/kaggle/

In [10]:
# ============================================================
# 4. CNN feature extraction with EfficientNet-B0
# ============================================================

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🖥 Using device:", device)

# Pretrained EfficientNet-B0
efficientnet = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
# Remove classifier head → keep global pooled features
efficientnet.classifier = torch.nn.Identity()
efficientnet.to(device)
efficientnet.eval()

# Standard ImageNet transforms
img_size = 224
cnn_transform = T.Compose([
    T.Resize((img_size, img_size)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])


class CSIROImageDataset(Dataset):
    def __init__(self, df, image_col="image_abs_path", transform=None):
        self.df = df.reset_index(drop=True)
        self.image_col = image_col
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.loc[idx, self.image_col]
        exists = os.path.exists(img_path) if isinstance(img_path, str) and img_path != "" else False

        if exists:
            try:
                image = Image.open(img_path).convert("RGB")
            except Exception:
                # In case of corrupted image, use blank
                image = Image.new("RGB", (img_size, img_size), (0, 0, 0))
                exists = False
        else:
            # Missing image → use blank image
            image = Image.new("RGB", (img_size, img_size), (0, 0, 0))

        if self.transform:
            image = self.transform(image)

        return image, exists


def extract_cnn_features(df, batch_size=32, image_col="image_abs_path"):
    """
    Returns a numpy array of shape (len(df), feature_dim).
    Missing images → zero-vector.
    """
    dataset = CSIROImageDataset(df, image_col=image_col, transform=cnn_transform)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    n_samples = len(df)
    example_input = torch.randn(1, 3, img_size, img_size).to(device)
    with torch.no_grad():
        example_output = efficientnet(example_input)
    feat_dim = example_output.shape[1]
    print(f"📐 CNN feature dimension: {feat_dim}")

    features = np.zeros((n_samples, feat_dim), dtype=np.float32)

    print(f"\n🖼 Extracting CNN features for {n_samples} images...")
    idx_start = 0
    for images, exists_flags in tqdm(loader):
        bs = images.size(0)
        images = images.to(device)

        with torch.no_grad():
            feats = efficientnet(images)  # (bs, feat_dim)

        # Convert to CPU numpy
        feats = feats.cpu().numpy()

        # For missing images, we already have zeros in 'features'
        # but here we overwrite everything; we need a mask
        for i in range(bs):
            if exists_flags[i]:
                features[idx_start + i] = feats[i]
            # else: keep zeros

        idx_start += bs

    print("✅ CNN features extracted with shape:", features.shape)
    return features


# Only run CNN if we actually have a working root/pattern with some images
if best_root is not None:
    
    test_cnn_feats  = extract_cnn_features(test_df,  batch_size=32)
    train_cnn_feats = extract_cnn_features(train_df, batch_size=32)

    # Attach to dataframes
    feat_cols = [f"img_feat_{i}" for i in range(train_cnn_feats.shape[1])]
    train_img_df = pd.DataFrame(train_cnn_feats, columns=feat_cols, index=train_df.index)
    test_img_df  = pd.DataFrame(test_cnn_feats,  columns=feat_cols, index=test_df.index)

    train_df = pd.concat([train_df, train_img_df], axis=1)
    test_df  = pd.concat([test_df,  test_img_df],  axis=1)

    print("✅ Fused CNN features into train_df and test_df.")
else:
    print("⚠️ No valid image root found → proceeding with TABULAR ONLY.")


🖥 Using device: cpu


Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:00<00:00, 165MB/s]


📐 CNN feature dimension: 1280

🖼 Extracting CNN features for 5 images...


100%|██████████| 1/1 [00:00<00:00,  1.06it/s]


✅ CNN features extracted with shape: (5, 1280)
📐 CNN feature dimension: 1280

🖼 Extracting CNN features for 1785 images...


100%|██████████| 56/56 [02:10<00:00,  2.33s/it]

✅ CNN features extracted with shape: (1785, 1280)
✅ Fused CNN features into train_df and test_df.


In [11]:
# ============================================================
# 5. Tabular preprocessing + feature engineering
# ============================================================

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer

# ---- Identify numeric columns from original training data ----
num_cols = train_df.select_dtypes(include=np.number).columns.tolist()
# Exclude target from num_cols for imputation
if "target" in num_cols:
    num_cols.remove("target")

print("Numeric columns (excluding target):", num_cols)

# ---- Concatenate train & test for consistent processing ----
train_for_concat = train_df.drop(columns=["target"], errors="ignore").copy()
full = pd.concat([train_for_concat, test_df], axis=0, ignore_index=True)

# ---- Impute numeric columns with median (fit on training data) ----
imputer = SimpleImputer(strategy="median")
imputer.fit(train_df[num_cols])
full[num_cols] = imputer.transform(full[num_cols])
print(f"✅ Imputed missing values in numerical columns: {', '.join(num_cols)} using median from training data.")

# ---- Feature Engineering: interaction term ----
if "Height_Ave_cm" in full.columns and "Pre_GSHH_NDVI" in full.columns:
    full["Height_x_NDVI"] = full["Height_Ave_cm"] * full["Pre_GSHH_NDVI"]
    print("✅ Created interaction feature 'Height_x_NDVI'")
else:
    print("⚠️ Could not create interaction feature (Height_Ave_cm or Pre_GSHH_NDVI missing).")

# ---- Polynomial features for Height_Ave_cm & Pre_GSHH_NDVI ----
poly_features_to_transform = []
if "Height_Ave_cm" in full.columns:
    poly_features_to_transform.append("Height_Ave_cm")
if "Pre_GSHH_NDVI" in full.columns:
    poly_features_to_transform.append("Pre_GSHH_NDVI")

if poly_features_to_transform:
    try:
        poly = PolynomialFeatures(degree=2, include_bias=False)
        poly_features = poly.fit_transform(full[poly_features_to_transform])
        poly_feature_names = poly.get_feature_names_out(poly_features_to_transform)
        poly_df = pd.DataFrame(poly_features, columns=poly_feature_names, index=full.index)

        # Keep only NEW polynomial terms (avoid duplicates)
        cols_to_add = [c for c in poly_df.columns if c not in poly_features_to_transform]
        poly_df = poly_df[cols_to_add]

        full = pd.concat([full, poly_df], axis=1)
        print(f"✅ Created polynomial features: {', '.join(cols_to_add)}")
    except Exception as e:
        print(f"❌ Error creating polynomial features: {e}")
else:
    print("⚠️ Skipping polynomial features; required columns missing.")


Numeric columns (excluding target): ['Pre_GSHH_NDVI', 'Height_Ave_cm', 'img_feat_0', 'img_feat_1', 'img_feat_2', 'img_feat_3', 'img_feat_4', 'img_feat_5', 'img_feat_6', 'img_feat_7', 'img_feat_8', 'img_feat_9', 'img_feat_10', 'img_feat_11', 'img_feat_12', 'img_feat_13', 'img_feat_14', 'img_feat_15', 'img_feat_16', 'img_feat_17', 'img_feat_18', 'img_feat_19', 'img_feat_20', 'img_feat_21', 'img_feat_22', 'img_feat_23', 'img_feat_24', 'img_feat_25', 'img_feat_26', 'img_feat_27', 'img_feat_28', 'img_feat_29', 'img_feat_30', 'img_feat_31', 'img_feat_32', 'img_feat_33', 'img_feat_34', 'img_feat_35', 'img_feat_36', 'img_feat_37', 'img_feat_38', 'img_feat_39', 'img_feat_40', 'img_feat_41', 'img_feat_42', 'img_feat_43', 'img_feat_44', 'img_feat_45', 'img_feat_46', 'img_feat_47', 'img_feat_48', 'img_feat_49', 'img_feat_50', 'img_feat_51', 'img_feat_52', 'img_feat_53', 'img_feat_54', 'img_feat_55', 'img_feat_56', 'img_feat_57', 'img_feat_58', 'img_feat_59', 'img_feat_60', 'img_feat_61', 'img_feat

/tmp/ipykernel_47/2667440366.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full["Height_x_NDVI"] = full["Height_Ave_cm"] * full["Pre_GSHH_NDVI"]


In [12]:
# ============================================================
# 6. Categorical encoding (avoiding image_path, IDs, dates)
# ============================================================

# Re-identify categoricals
cat_cols_after_fe = full.select_dtypes(include=["object", "category"]).columns.tolist()

# We explicitly DO NOT encode these (they are IDs / paths / dates)
cols_to_exclude_from_cat = ["target", "image_path", "sample_id", "Sampling_Date"]

cat_cols_for_encoding = [
    c for c in cat_cols_after_fe
    if c not in cols_to_exclude_from_cat
]

print("Categorical columns to one-hot encode:", cat_cols_for_encoding)

full_encoded = pd.get_dummies(full, columns=cat_cols_for_encoding, drop_first=False)
print("✅ One-hot encoded categorical columns.")

# ---- Split back into train & test ----
train_encoded = full_encoded.iloc[: len(train_df)].copy()
test_encoded  = full_encoded.iloc[len(train_df):].copy()

# Re-attach target
target_col = "target"
if target_col in train_df.columns:
    train_encoded[target_col] = train_df[target_col].values
else:
    raise ValueError(f"Expected '{target_col}' in original training data.")

# Drop non-feature/ID columns from X
drop_cols = [c for c in ["sample_id", "image_path", "Sampling_Date", "target_name"] if c in train_encoded.columns]
X = train_encoded.drop(columns=drop_cols + [target_col], errors="ignore")
y = train_encoded[target_col]

# Align test columns with training features
X_test_final = test_encoded.drop(columns=drop_cols, errors="ignore").reindex(columns=X.columns, fill_value=0)

# Train/validation split
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

print("\nData shapes after preprocessing, CNN fusion and FE:")
print("X      :", X.shape)
print("X_train:", X_train.shape, "| X_valid:", X_valid.shape)
print("X_test :", X_test_final.shape)
print("y      :", y.shape)


Categorical columns to one-hot encode: ['State', 'Species', 'target_name', 'image_abs_path']
✅ One-hot encoded categorical columns.

Data shapes after preprocessing, CNN fusion and FE:
X      : (1785, 1668)
X_train: (1428, 1668) | X_valid: (357, 1668)
X_test : (5, 1668)
y      : (1785,)


In [2]:
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from sklearn.metrics import r2_score


ModuleNotFoundError: No module named 'lightgbm'

In [13]:
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from sklearn.metrics import r2_score

print("\n⚡ Training Faster LightGBM model with early stopping...")

# Optional: make sure data is float32 for a small speed boost
X_train_lgb = X_train.astype("float32")
X_valid_lgb = X_valid.astype("float32")
X_full_lgb  = X.astype("float32")

lgbm_fast = LGBMRegressor(
    # lower upper bound; we want ES to stop well before this
    n_estimators=1200,
    learning_rate=0.05,      # a bit larger → fewer trees needed
    num_leaves=48,           # slightly smaller trees
    max_depth=8,             # cap depth → faster per tree
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_samples=20,
    reg_lambda=1.0,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=-1,
)

lgbm_fast.fit(
    X_train_lgb, y_train,
    eval_set=[(X_valid_lgb, y_valid)],
    eval_metric="rmse",
    callbacks=[
        early_stopping(stopping_rounds=50),   # stop earlier
        log_evaluation(period=0),             # no spam in logs
    ],
)

best_iter = lgbm_fast.best_iteration_ or lgbm_fast.n_estimators
print(f"\n✅ Best iteration (fast model): {best_iter}")

valid_pred = lgbm_fast.predict(X_valid_lgb)
valid_r2 = r2_score(y_valid, valid_pred)
print("📊 Fast LightGBM Validation R2:", valid_r2)

# ---------- final_model on FULL data ----------
final_model = LGBMRegressor(
    n_estimators=best_iter,
    learning_rate=0.05,
    num_leaves=48,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_samples=20,
    reg_lambda=1.0,
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbose=-1,
)

final_model.fit(X_full_lgb, y)
print("✅ final_model (Fast LightGBM) trained on FULL training data.")



⚡ Training Faster LightGBM model with early stopping...
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1179]	valid_0's rmse: 9.90539	valid_0's l2: 98.1167

✅ Best iteration (fast model): 1179
📊 Fast LightGBM Validation R2: 0.8542686927583144
✅ final_model (Fast LightGBM) trained on FULL training data.


In [14]:
# ============================================================
# 7. Predict on TEST set and create submission.csv
# ============================================================

# 1. Make test features float32 like training
X_test_lgb = X_test_final.astype("float32")

# 2. Predict with the final_model
test_pred = final_model.predict(X_test_lgb)

# 3. Build submission in LONG format
#    - one row per (image, target_name)
#    - sample_id is already in test_df and matches sample_submission format
submission = pd.DataFrame({
    "sample_id": test_df["sample_id"],
    "target": test_pred.astype("float32")
})

# 4. Save with the required filename
submission.to_csv("submission.csv", index=False)

print(submission.head())
print("Submission shape:", submission.shape)


                    sample_id     target
0  ID1001187975__Dry_Clover_g   3.768795
1    ID1001187975__Dry_Dead_g  29.119774
2   ID1001187975__Dry_Green_g  23.413101
3   ID1001187975__Dry_Total_g  52.545574
4         ID1001187975__GDM_g  26.124210
Submission shape: (5, 2)


In [1]:
import joblib
import os

os.makedirs("artifacts", exist_ok=True)

joblib.dump(lightgbm, "artifacts/model.joblib")

# Only if you used a preprocessor
joblib.dump(preprocessor, "artifacts/preprocessor.joblib")

print("Model exported successfully")


NameError: name 'lightgbm' is not defined